# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

1    855
0    744
Name: credit_ranking, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column
y=loans_df['credit_ranking']
# Display a sample of y
y.head()

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [6]:
# Define features set X by selecting all columns but credit_ranking
X=loans_df.drop(columns=['credit_ranking'])
# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [7]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [8]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
# Fit the scaler to the features testing dataset
X_test_scaled = scaler.transform(X_test)
# Display the first three few entries of the scaled features
print("X_train_scaled sample:\n", X_train_scaled[:3])
print("X_test_scaled sample:\n", X_test_scaled[:3])

X_train_scaled sample:
 [[-0.73307913  0.6648928  -1.25704443 -0.3204585  -0.45362151 -0.74240736
  -0.6455073   0.24000129  0.98846046  0.0630946  -0.87223395]
 [ 1.06774091 -0.62346154  1.52314768  0.60886277 -0.36954631 -1.12518952
  -1.11200285  0.18789883 -1.7535127  -0.17390392 -0.77978452]
 [-1.74604041 -1.07158479 -1.35814232 -0.53491726 -0.78992229  1.07580793
   0.53628144 -2.67773653  2.32756363  0.77409018  3.28799021]]
X_test_scaled sample:
 [[ 0.27988214 -0.67947695  1.87699031 -0.24897225  0.11388606 -0.64671182
  -0.11681233  0.96943574  1.43482818  0.00384497  0.05226031]
 [ 0.22360652  0.55286199  0.05722821  0.10845901  0.17694246  0.11885252
   0.72287966  0.31294474 -0.35064271 -0.17390392 -0.22508797]
 [ 1.18029216 -1.07158479  1.57369663  0.82332153  1.81640878 -0.93379844
  -0.92540463  0.13579637 -0.79701044  0.2408435   0.79185571]]


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [9]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
display ("Number of input features:",input_nodes)

'Number of input features:'

11

In [10]:
# Define the number of hidden nodes for the first hidden layer
hidden_units_layer1 = 64

# Define the input layer
input_layer = tf.keras.layers.Input(shape=(input_nodes,))

# Define the first hidden layer with ReLU activation
hidden_layer1 = tf.keras.layers.Dense(hidden_units_layer1, activation='relu')(input_layer)


In [11]:
# Define the number of hidden nodes for the first hidden layer
hidden_units_layer1 = 64

# Define the number of hidden nodes for the second hidden layer
hidden_units_layer2 = 32

# Define the number of neurons in the output layer
output_neurons = 1 

In [45]:
# Create the Sequential model instance
mod = Sequential()

# Add the first hidden layer
mod.add(Dense(units=hidden_units_layer1, activation='relu', input_dim=input_nodes))

# Add the second hidden layer
mod.add(Dense(units=hidden_units_layer2, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
mod.add(Dense(units=output_neurons, activation='sigmoid'))


In [46]:
# Display the Sequential model summary
mod.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [47]:
# Compile the Sequential model
mod.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [48]:
# Fit the model using 50 epochs and the training data
history = mod.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6784 - loss: 0.6217 - val_accuracy: 0.7734 - val_loss: 0.5561
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7357 - loss: 0.5528 - val_accuracy: 0.7695 - val_loss: 0.5279
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7570 - loss: 0.5122 - val_accuracy: 0.7656 - val_loss: 0.5201
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7553 - loss: 0.5025 - val_accuracy: 0.7578 - val_loss: 0.5099
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7632 - loss: 0.4966 - val_accuracy: 0.7539 - val_loss: 0.5071
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7665 - loss: 0.4806 - val_accuracy: 0.7578 - val_loss: 0.5043
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7611 - loss: 0.4839 - val_accuracy: 0.7539 - val_loss: 0.5011
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7691 - loss: 0.4713 - val_accuracy: 0.7500 - val_loss:

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [49]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
loss, accuracy = mod.evaluate(X_test_scaled, y_test, verbose=2)

# Display evaluation results
print(f"Loss: {loss}, Accuracy: {accuracy}")

10/10 - 0s - 2ms/step - accuracy: 0.7563 - loss: 0.5805
Loss: 0.5804818868637085, Accuracy: 0.7562500238418579


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [50]:
# Set the model's file path
model_path = 'student_loan_mod.keras'

# Export your model to a keras file
mod.save(model_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [51]:
# Set the model's file path
model_path = 'student_loan_mod.keras'

# Load the model to a new object
nn_import = tf.keras.models.load_model(model_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [52]:
# Make predictions with the test data
pred = mod.predict(X_test_scaled)

# Display a sample of the predictions
print('Sample predictions:')
print(pred[:5])

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 610us/step
Sample predictions:
[[0.12771696]
 [0.38973677]
 [0.9035951 ]
 [0.80340433]
 [0.98371094]]


In [53]:
# Save the predictions to a DataFrame and round the predictions to binary results
pred_df = pd.DataFrame(columns=["predictions"], data=pred)
pred_df["predictions"] = round(pred_df["predictions"], 0)

# Review the DataFrame
pred_df

,predictions
0,0.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
315,1.0
316,0.0
317,0.0
318,1.0


### Step 4: Display a classification report with the y test data and predictions

In [55]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, pred_df['predictions']))

              precision    recall  f1-score   support

           0       0.75      0.75      0.75       154
           1       0.77      0.77      0.77       166

    accuracy                           0.76       320
   macro avg       0.76      0.76      0.76       320
weighted avg       0.76      0.76      0.76       320



---
## Discuss creating a recommendation system for student loans

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**
 
To build a recommendation system for student loan options, the following data is essential:
1. Student Demographics: Age, gender, location, family income, and parental education levels.
2. Academic Information: GPA, major, school ranking, and expected graduation date.
3. Loan History: Previous loans taken, repayment history, and loan amounts.
4. Financial Aid Information: Scholarships, grants, work-study programs, and other financial aid received.
5. Employment Information: Part-time jobs, internships, and job offers.
6. Preferences and Needs: Preferences for loan types (fixed vs. variable rates), repayment terms, and    deferment options.

This data provides a comprehensive picture of the student's financial situation, academic prospects, and borrowing behavior, which are critical for recommending appropriate loan options.

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

The recommendation system would primarily use content-based filtering, which focuses on the attributes of the loans and the user's profile. Content-Based filtering allows for the data to include detailed attributes about students and loans, such as academic performance, financial status, and personal preferences. Each recommendation can be personalized based on the student's specific profile and needs. It also allows the system to recommend loans that fit the student's unique circumstances, even if similar users have not taken those loans. Collaborative filtering could complement this if there's enough data on student interactions with loan options, but content-based filtering is more suitable given the detailed and varied attributes involved.

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

First Example: 
Bias and Fairness Concern: The system must avoid biases that could unfairly affect certain student groups, like those from lower-income families.

First Solution: Regularly check for biases, use diverse training data, and include fairness rules in the algorithms. This helps maintain trust and ensures fair loan recommendations for all students.

Second Example: Data Privacy and Security Concern: Student loan applications involve sensitive personal and financial information. Ensuring this data is protected is crucial to prevent identity theft and fraud.

Second Solution: Use strong data encryption, secure access protocols, and comply with data protection regulations like GDPR or FERPA to keep student information safe.
